# Assignment2

In [1]:
import numpy as np
import copy as copy
import matplotlib.pyplot as plt
from scipy.linalg import norm
import math 
import matplotlib.gridspec as gridspec
from scipy.interpolate import interp2d

f1=open("xc.dat",'r')
f2=open("yc.dat")
f3=open("u.dat")
f4=open("v.dat")



xc=f1.readlines()
xc = list(map(float, xc))
xc=np.array(xc)
#print(xc)
nim1=len(xc)
ni=nim1+1

yc=f2.readlines()
yc = list(map(float, yc))
yc=np.array(yc)
njm1=len(yc) #26
nj=njm1+1 #27

u=f3.readlines()
u = list(map(float, u))
u2d=np.reshape(u,(ni,nj)) #27*27
u2d=np.array(u2d)
u2d=u2d.transpose()

v=f4.readlines()
v = list(map(float, v))
v2d=np.reshape(v,(ni,nj)) #27*27
v2d=np.array(v2d)
v2d=v2d.transpose()
#print(v2d.shape)


#  xc and yc are the coordinates of the grid

#        o---------------o  yc(j)
#        |               |
#        |               |
#        |       P       |
#        |               |
#        |               |
#        o---------------o  yc(j-1)

#      xc(i-1)          xc(i)          

# The cell-node (P) has the coordinates xp(i), yp(j)

# compute the x-coordinates of the cell centres.........................
xp= np.ones(nim1+1)
for i in range(1,nim1):
    xp[i]=0.5*(xc[i]+xc[i-1])
xp[0]=xc[0]
xp[-1]=xc[-1]

yp= np.ones(njm1+1)
for i in range(1,njm1):
    yp[i]=0.5*(yc[i]+yc[i-1])
yp[0]=yc[0]
yp[-1]=yc[-1]
#u for the faces:................................................
u=np.zeros((nim1,njm1),dtype=np.float32)
v=np.zeros((nim1,njm1),dtype=np.float32)
for i in range(1,nim1):
    for j in range(1,njm1):
        u[i][j]=0.5*(u2d[i][j]+u2d[i][j+1])
        u[0][j]=u2d[0][j]
        u[i][0]=u2d[i][0]
for i in range(1,nim1):
    for j in range(1,njm1):
        v[i][j]=0.5*(v2d[i][j]+v2d[i][j+1])
        v[0][j]=v2d[0][j]
        v[i][0]=v2d[i][0]
#d_x and d_y calculation distane between cell center + 2 DISTANCE FOR BOUNDARY CELL .................
d_x=np.zeros(nim1)
d_y=np.zeros(njm1)
for i in range(0,nim1):
    d_x[i]= xp[i+1]-xp[i]
for i in range(0,njm1):
    d_y[i]= yp[i+1]-yp[i]
#calculation for del_x and del_y
del_x=np.zeros(nim1-1)
del_y=np.zeros(njm1-1)
for i in range(0,nim1-1):
    del_x[i]= xc[i+1]-xc[i]
for i in range(0,njm1-1):
    del_y[i]= yc[i+1]-yc[i]

#.............................................write the convection Diffusion code..............................
Ta=20
T2=10
gamma=1/500 #k_by_cp
#.................................................................................
N=Nx*Ny
key=0
epc=10**(-5)
T_0=np.ones((len(xp),len(yp)),dtype=np.float32)
T=np.ones((len(xp),len(yp)),dtype=np.float32)
residu=[]
itaration=[]
count=0
j=0
while(key==0):
    for i in range(1,len(yp)-1): #till 25 1.9
        for j in range(1,len(xp)-1):
            if(i==1 and j==1): #1
                Fw=rho*u[j-1][i-1]*(yc[i-1]-yc[i])
                Fe=rho*u[j][i-1]*(yc[i-1]-yc[i])
                Fs=rho*v2d[j-1][i-1]*
                Fn=rho*v2d[j-1][]*
                Sp=-((2*ks*dx[j])/dy[i])+(-1.5*dx[j]*dy[i])/T_0[i][j]                      
                Su=((2*ks*dx[j])/dy[i])*T1                
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+An*T[i+1][j]+Su)/Ap
            elif(i==0 and j==Nx-1):#3
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                T2=(-5*node_y[i])+15*math.cos(2*math.pi*node_y[i]/H)
                Aw=(kw*dy[i])/(node_x[j]-node_x[j-1])
                Ae=0
                An=(kn*dx[j])/(node_y[i+1]-node_y[i])
                As=0
                Sp=-((2*ks*dx[j])/dy[i])-((2*ke*dy[i])/dx[j])+(-1.5*dx[j]*dy[i])/T_0[i][j]                           
                Su=((2*ks*dx[j])/dy[i])*T1+((2*ke*dy[i])/dx[j])*T2                          
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Aw*T[i][j-1]+An*T[i+1][j]+Su)/Ap
            elif(i==Ny-1 and j==Nx-1):#9
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                T2=(-5*node_y[i])+15*math.cos(2*math.pi*node_y[i]/H)
                Aw=(kw*dy[i])/(node_x[j]-node_x[j-1])
                Ae=0
                An=0
                As=(ks*dx[j])/(node_y[i]-node_y[i-1])
                Sp=-((2*kn*dx[j])/dy[i])-((2*ke*dy[i])/dx[j])+(-1.5*dx[j]*dy[i])/T_0[i][j]                    
                Su=+((2*kn*dx[j])/dy[i])*T3+((2*ke*dy[i])/dx[j])*T2             
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Aw*T[i][j-1]+As*T[i-1][j]+Su)/Ap
            elif(i==Ny-1 and j==0):#7done
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                Aw=0
                Ae=(ke*dy[i])/(node_x[j+1]-node_x[j])
                An=0
                As=(ks*dx[j])/(node_y[i]-node_y[i-1])
                Sp=-((2*kn*dx[j])/dy[i])+(-1.5*dx[j]*dy[i])/T_0[i][j]                    
                Su=+((2*kn*dx[j])/dy[i])*T3
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+As*T[i-1][j]+Su)/Ap
            elif(0<j<(Nx-1) and i==0):   #2done
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                Aw=(kw*dy[i])/(node_x[j]-node_x[j-1])
                Ae=(ke*dy[i])/(node_x[j+1]-node_x[j])
                An=(kn*dx[j])/(node_y[i+1]-node_y[i])
                As=0
                Sp=-((2*ks*dx[j])/dy[i])+(-1.5*dx[j]*dy[i])/T_0[i][j]  
                Su=((2*ks*dx[j])/dy[i])*T1
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+Aw*T[i][j-1]+An*T[i+1][j]+Su)/Ap
            elif(j==Nx-1 and 0<i<(Ny-1)):#6done
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                T2=(-5*node_y[i])+15*math.cos(2*math.pi*node_y[i]/H)
                Aw=(kw*dy[i])/(node_x[j]-node_x[j-1])
                Ae=0
                An=(kn*dx[j])/(node_y[i+1]-node_y[i])
                As=(ks*dx[j])/(node_y[i]-node_y[i-1])
                Sp=-((2*ke*dy[i])/dx[j]) +(-1.5*dx[j]*dy[i])/T_0[i][j] 
                Su=((2*ke*dy[i])/dx[j])*T2 
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(An*T[i+1][j]+As*T[i-1][j]+Aw*T[i][j-1]+Su)/Ap
            elif(0<j<(Nx-1) and i==(Ny-1)):#8
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                Aw=(kw*dy[i])/(node_x[j]-node_x[j-1])
                Ae=(ke*dy[i])/(node_x[j+1]-node_x[j])
                An=0
                As=(ks*dx[j])/(node_y[i]-node_y[i-1])
                Sp=-((2*kn*dx[j])/dy[i])+(-1.5*dx[j]*dy[i])/T_0[i][j]                    
                Su=+((2*kn*dx[j])/dy[i])*T3                
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+Aw*T[i][j-1]+As*T[i-1][j]+Su)/Ap
            elif(0<i<(Ny-1) and j==0):#4
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                Aw=0
                Ae=(ke*dy[i])/(node_x[j+1]-node_x[j])
                An=(kn*dx[j])/(node_y[i+1]-node_y[i])
                As=(ks*dx[j])/(node_y[i]-node_y[i-1])
                Sp=(-1.5*dx[j]*dy[i])/T_0[i][j]
                Su=0
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+As*T[i-1][j]+An*T[i+1][j]+Su)/Ap
            else:                            #(i!=0 and i!=(Ny-1) and j!=0 and j!=(Nx-1) ):#5
                ke=5*(1+(100*cx[j+1])/L)
                kw=5*(1+(100*cx[j])/L)
                kn=ks=5*(1+(100*node_x[j])/L)
                Aw=(kw*dy[i])/(node_x[j]-node_x[j-1])
                Ae=(ke*dy[i])/(node_x[j+1]-node_x[j])
                An=(kn*dx[j])/(node_y[i+1]-node_y[i])
                As=(ks*dx[j])/(node_y[i]-node_y[i-1])
                Sp=(-1.5*dx[j]*dy[i])/T_0[i][j]
                Su=0
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+Aw*T[i][j-1]+An*T[i+1][j]+As*T[i-1][j]+Su)/Ap
    #.........................END OF FOR LOOP.........................................
    #.......................................Calculate Resu:........................................................
    count=count+1
    itaration.append(count)            #itaration[j]=count
    #R[j]=(norm(T-T_0))/len(T)
    R=(norm(T-T_0))/len(T)
    residu.append(R)
    if(R<epc):
        key=1
    T_0=copy.copy(T)







#..................plot the Graph.....................................
[m1,m2]=np.meshgrid(xc,yc) #to view mesh plot
plt.plot(m1,yc)
plt.plot(xc,np.transpose(m2))
plt.savefig("output1.png",dpi=600)
plt.show()

# plt.quiver(xp,yp,u2d,v2d,9)
# plt.savefig("output2.png",dpi=600)
# plt.show()

#...................close the Files................................
f1.close()
f2.close()
f3.close()
f4.close()

SyntaxError: invalid syntax (201569219.py, line 109)

In [149]:
print(yc.shape)
print(yp.shape)

(26,)
(27,)


In [160]:
len(xp)

27

In [164]:
xp[26]

2.0